In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install xmltodict

In [ ]:
!ls

 1.jpg				   folder
 augmentation_configuration.json   images
 cars_dataset			   impy-0.1-py3-none-any.whl
'Colab Notebooks'		   impy-0.1-py3-none-any.whl.1
 corped_images.ipynb		   impy-0.1-py3-none-any.whl.2
 data				   yolov5


In [ ]:
import glob
import os
import xmltodict
import json
import pprint

from PIL import Image
import numpy as np
import sys

import csv

In [ ]:
# default format can be changed as needed
def createFileList(myDir, format='.jpg'):
    fileList = []
    print(myDir)
    for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList


In [ ]:
# Printing results
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
# Look for XML files and parses then as if they were Pascal VOC Files
def process():
    # Finds all XML files on data/ and append to list
    pascal_voc_contents = []
    os.chdir("data")

    print("Found {} files in data directory!".format(
        str(len(glob.glob("*.xml")))))
    for file in glob.glob("*.xml"):
        f_handle = open(file, 'r')
        print("Parsing file '{}'...".format(file))
        pascal_voc_contents.append(xmltodict.parse(f_handle.read()))

    # Process each file individually
    for index in pascal_voc_contents:
        image_file = index['annotation']['filename']
        # If there's a corresponding file in the folder,
        # process the images and save to output folder
        if os.path.isfile(image_file):
            extractDataset(index['annotation'])
        else:
            print("Image file '{}' not found, skipping file...".format(image_file))


In [ ]:
# Extract image samples and save to output dir
def extractDataset(dataset):
    print("Found {} objects on image '{}'...".format(
        len(dataset['object']), dataset['filename']))

    # Open image and get ready to process
    img = Image.open(dataset['filename'])

    # Create output directory
    save_dir = dataset['filename'].split('.')[0]
    try:
        os.mkdir(save_dir)
    except:
        pass
    # Image name preamble
    sample_preamble = save_dir + "/" + dataset['filename'].split('.')[0] + "_"
    # Image counter
    i = 0

    # Run through each item and save cut image to output folder
    for item in dataset['object']:
        # Convert str to integers
        bndbox = dict([(a, int(b)) for (a, b) in item['bndbox'].items()])
        # Crop image
        img_file = img.crop((bndbox['xmin'], bndbox['ymin'],
                       bndbox['xmax'], bndbox['ymax']))
        # Save
        img_file.save(sample_preamble + str(i) + '.jpg')

        #saving as csv form
        width, height = img_file.size
        format = img_file.format
        mode = img_file.mode

        img_grey = img_file.convert('L')
        value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((img_grey.size[1], img_grey.size[0]))
        value = value.flatten()
        print(value)
        with open("/content/drive/MyDrive/data/csv/images.csv", 'a') as f:
          writer = csv.writer(f)
          writer.writerow(value)
        i = i + 1


In [ ]:
if __name__ == '__main__':
  process()

Found 5 files in data directory!
Parsing file 'cars0.xml'...
Parsing file 'cars1.xml'...
Parsing file 'cars2.xml'...
Parsing file 'cars3.xml'...
Parsing file 'cars4.xml'...
Found 16 objects on image 'cars0.png'...
[88 95 97 ... 15 15 15]
[254 254 254 ... 136 135 135]
[23 24 19 ... 67 67 67]
[20 20 23 ... 63 63 63]
[ 5  4  4 ... 67 66 66]
[25 25 25 ... 78 78 78]
[74 74 73 ... 79 77 77]
[69 52 41 ... 46 46 46]
[ 5  5  5 ... 76 75 75]
[ 33  32  30 ... 161 163 163]
[ 65  68  71 ... 113 113 113]
[31 24 17 ... 64 65 67]
[34 34 34 ... 19 19 19]
[19 18 18 ... 67 67 67]
[23 22 22 ... 86 87 86]
[27 29 30 ... 25 23 22]
Found 15 objects on image 'cars1.png'...
[189 188 188 ... 128 127 125]
[217 217 218 ... 190 189 187]
[127 127 126 ... 182 185 188]
[196 197 196 ... 173 171 160]
[ 73  77  81 ... 196 195 196]
[41 42 42 ... 93 95 98]
[ 40  40  38 ... 204 204 204]
[195 195 195 ... 178 178 179]
[145 146 146 ... 170 170 170]
[213 213 213 ... 189 183 173]
[ 47  47  46 ... 217 217 217]
[107  81  59 ... 10

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/data/csv/images.csv")
print(df.shape)
#df.head()